In [153]:
import pandas as pd 
import ast
import re
import numpy as np
import nltk
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize, TreebankWordTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.util import ngrams
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer


In [154]:
df = pd.read_csv("./data/Kickstarter_merged.csv")
df.head(3)
df_copy = df.copy()

In [155]:
df_copy.describe()

,id,pledged,goal,update_count,backers_count,is_starrable,spotlight,staff_pick
count,2.132000e+03,2132.000000,2.132000e+03,2132.000000,2132.000000,2132.000000,2132.000000,2132.000000
mean,1.077279e+09,13784.319747,6.604620e+04,5.312383,119.403846,0.009381,0.432458,0.045966
std,6.225447e+08,41074.680798,2.166310e+06,8.216737,547.130104,0.096422,0.495533,0.209461
min,1.454620e+06,0.000000,2.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
25%,5.370663e+08,147.750000,1.200000e+03,0.000000,4.000000,0.000000,0.000000,0.000000
50%,1.105835e+09,1594.210000,5.000000e+03,2.000000,23.000000,0.000000,0.000000,0.000000
75%,1.622623e+09,9547.250000,1.500000e+04,8.000000,88.000000,0.000000,1.000000,0.000000
max,2.145266e+09,515413.000000,1.000000e+08,67.000000,18768.000000,1.000000,1.000000,1.000000


In [157]:
df_copy.head()

,id,name,description,description_story,description_risks,rewards,category,pledged,goal,deadline,...,spotlight,staff_pick,video,creator_name,creator_url,url,created_at,published_at,launched_at,link
0,1249154571,Bunny Care Clinic Pin and Apparel Collection,A small collection of Bunny themed enamel pins...,Story\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\...,We try our best to keep everything on schedule...,"[{'rewards': 'Pledge without a reward'}, {'rew...",art/illustration,14115.0,700,2022-10-04 01:25:20,...,0,0,NaN,Labutori,https://www.kickstarter.com/profile/labutori,https://www.kickstarter.com/projects/labutori/...,2022-05-25 03:28:55+00:00,2022-09-09 01:25:20+00:00,2022-09-09 01:25:20+00:00,https://www.kickstarter.com/projects/labutori/...
1,1276054891,Hustle: A Singaporean Card Game,Hustle: A Singaporean Card Game is a funny and...,Story\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\...,We want to be frank and honest with all our ge...,"[{'rewards': 'Pledge without a reward'}, {'rew...",games/tabletop games,6.0,50,2022-10-07 13:08:05,...,0,0,https://v2.kickstarter.com/1662721387-XBw1i2Sj...,Hustle Singapore,https://www.kickstarter.com/profile/hustlesg,https://www.kickstarter.com/projects/hustlesg/...,2022-08-20 09:52:01+00:00,2022-09-07 13:08:05+00:00,2022-09-07 13:08:05+00:00,https://www.kickstarter.com/projects/hustlesg/...
2,236207086,"Neovide, Waterless One-Stop Sous Vide Cooker",No more water containers and vacuum bags. With...,Story\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\...,"With our years of experience with products, ou...","[{'rewards': 'Pledge without a reward'}, {'rew...",technology,289082.0,10000,2022-10-06 13:00:04,...,0,1,https://v2.kickstarter.com/1662723490-gvkAMr9s...,The Space Tech,https://www.kickstarter.com/profile/thespacetech,https://www.kickstarter.com/projects/thespacet...,2022-06-30 09:28:52+00:00,2022-09-06 13:00:04+00:00,2022-09-06 13:00:04+00:00,https://www.kickstarter.com/projects/thespacet...
3,2128144913,Lit Cafe,Little Toasts of Happiness,StoryHi! \r\nLit Cafe is a small space to prov...,The concept is to offer affordable local food ...,"[{'rewards': 'Pledge without a reward'}, {'rew...",food/spaces,170.0,12000,2022-10-02 13:09:00,...,0,0,NaN,Kay En,https://www.kickstarter.com/profile/hosum,https://www.kickstarter.com/projects/hosum/ho-...,2022-08-30 08:28:52+00:00,2022-09-06 04:29:02+00:00,2022-09-06 04:29:02+00:00,https://www.kickstarter.com/projects/hosum/ho-...
4,891970407,Runway Tarot & Golden Journey Tarot,When the fashion week come into Tarot.\r\nThis...,Story\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\...,COLLABORATION\r\nThis is the third time that b...,"[{'rewards': 'Pledge without a reward'}, {'rew...",art/painting,33599.0,6800,2022-10-05 13:57:35,...,0,0,https://v2.kickstarter.com/1662720057-aeWt13h6...,Eugene Leong,https://www.kickstarter.com/profile/locationtarot,https://www.kickstarter.com/projects/locationt...,2022-07-18 14:48:29+00:00,2022-09-05 13:57:35+00:00,2022-09-05 13:57:35+00:00,https://www.kickstarter.com/projects/locationt...


In [165]:
# process rewards string
def process_rewards(corpus):
    corpus_processed = []
    for row in corpus:
        row_processed = ""
        row.replace("\\n", " ")
        row = ast.literal_eval(row)

        for dict in row:
            row_processed += dict['rewards'].lower() + ' '
        
        row_processed = row_processed.replace("//",'')
        corpus_processed.append(row_processed)

    return corpus_processed

df_copy["rewards_processed"] = process_rewards( df_copy["rewards_list"])


In [166]:
df_copy["rewards_processed"]

0       pledge without a reward pledge s$ 12 or more  ...
1       pledge without a reward pledge s$ 2 or more  a...
2       pledge without a reward pledge s$ 2 or more  a...
3       pledge without a reward pledge s$ 7 or more  a...
4       pledge without a reward pledge s$ 2 or more  a...
                              ...                        
2127    pledge s$ 5 or more  about us$ 4    a thank yo...
2128    pledge s$ 40 or more  about $29    handmade by...
2129    pledge s$ 7 or more  about us$ 6    city rawke...
2130    pledge s$ 2 or more  about $2    every dollar ...
2131    pledge s$ 50 or more  about us$ 36    the onli...
Name: rewards_processed, Length: 2132, dtype: object

In [171]:
df_copy["description"]

0       A small collection of Bunny themed enamel pins...
1       Hustle: A Singaporean Card Game is a funny and...
2       No more water containers and vacuum bags. With...
3                              Little Toasts of Happiness
4       When the fashion week come into Tarot.\r\nThis...
                              ...                        
2127    Help me make flowers in Singapore affordable a...
2128    A wallet/card keeper, hand-folded from leather...
2129    Record App Studio features karaoke singing fro...
2130    Reinventing the romance genre where 'Happily E...
2131    The Online Deals is a location based discount ...
Name: description, Length: 2132, dtype: object

In [182]:
# process description story
def process_description_story(corpus):
    corpus_processed = []
    for row in corpus:
        row = str(row)
        row_processed = row.replace("\r", " " )
        row_processed = row_processed.replace("\n", " " )
        corpus_processed.append(row_processed)

    return corpus_processed

df_copy["description_story_processed"] = process_description_story(df_copy["description_story"])
df_copy["description_risks_processed"] = process_description_story(df_copy["description_risks"])

In [184]:
df_copy["description_risks_processed"][1]

'We want to be frank and honest with all our generous backers so that you get the most genuine experience with our product.      FUNDING   Kickstarter has an "All-or-Nothing" policy where the campaign gets $0 funding if the campaign doesn\'t reach its target. It would mean so much to the HUSTLE team if you could help to spread and share the word about this game so it becomes a reality for you (and for us).  The HUSTLE team genuinely believes in this game being able to start new conversations about what it means to be financially independent in Singapore in this day and age. The risk we are taking with Kickstarter is that if we are unable to be fully funded, we are not able to get the funding needed.'

In [128]:
X, y = df_copy[['id', 'name', 'description', 'description_story', 'description_risks',
       'rewards', 'category', 'pledged', 'goal', 'deadline', 'location',
       'state', 'faq_count', 'update_count', 'backers_count', 'is_starrable',
       'spotlight', 'staff_pick', 'video', 'creator_name', 'creator_url',
       'url', 'created_at', 'published_at', 'launched_at', 'link',
       'rewards_list', 'rewards_processed']], df_copy["state"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True)

In [129]:
X_train

,id,name,description,description_story,description_risks,rewards,category,pledged,goal,deadline,...,video,creator_name,creator_url,url,created_at,published_at,launched_at,link,rewards_list,rewards_processed
1528,1610812279,AIKO: The jumpsuit for every woman,AIKO is a sustainable jumpsuit brand focusing ...,StoryTHE INSPIRATION\r\n I grew up feeling ver...,Time is one of the biggest challenges for this...,[{'rewards': 'Pledge S$ 5 or more\n\nAbout US$...,fashion/apparel,136.0,2000,2018-05-16 16:29:00,...,https://v2.kickstarter.com/1662718144-kzAYmLow...,Kimberly Leong,https://www.kickstarter.com/profile/433705528,https://www.kickstarter.com/projects/433705528...,2018-04-26 14:46:56+00:00,2018-04-26 15:10:58+00:00,2018-04-26 15:10:58+00:00,https://www.kickstarter.com/projects/433705528...,[{'rewards': 'Pledge S$ 5 or more About US$ 4...,pledge s$ 5 or more about us$ 4 make a ple...
648,42628029,The Everyday Wallet - The only wallet you will...,Super slim minimalist and RFID blocking wallet...,StoryPLAIN SUPPLIES was proudly featured on\r\...,"At PLAIN SUPPLIES, we have a great manufacturi...",[{'rewards': 'Pledge S$ 2 or more\n\nAbout US$...,design/product design,10567.0,8000,2018-12-19 13:00:15,...,https://v2.kickstarter.com/1662645816-ym04zM24...,PLAIN SUPPLIES,https://www.kickstarter.com/profile/878251512,https://www.kickstarter.com/projects/878251512...,2018-09-08 06:10:09+00:00,2018-11-19 13:00:15+00:00,2018-11-19 13:00:15+00:00,https://www.kickstarter.com/projects/878251512...,[{'rewards': 'Pledge S$ 2 or more About US$ 2...,pledge s$ 2 or more about us$ 2 thank you!...
1441,352803866,Capital P - Sustainable patchwork garment,"CapitalP, It is a sustainable label that use u...",Story\r\n\r\n\r\nLogo\r\n\r\n\r\n\r\n\r\n\r\n\...,The biggest challenge is working as a designer...,[{'rewards': 'Pledge S$ 2 or more\n\nAbout US$...,fashion/apparel,2.0,800,2018-11-30 07:34:00,...,https://v2.kickstarter.com/1662717617-h4t4ElLS...,Patcharapol Soithong,https://www.kickstarter.com/profile/2065287686,https://www.kickstarter.com/projects/206528768...,2018-10-22 07:30:46+00:00,2018-10-22 07:58:02+00:00,2018-10-22 07:58:02+00:00,https://www.kickstarter.com/projects/206528768...,[{'rewards': 'Pledge S$ 2 or more About US$ 2...,pledge s$ 2 or more about us$ 2 thank you!...
2098,579693277,"Battery-Less, Automatic Watches Without the Ma...",The first automatic watch without the price ma...,Story\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n New to K...,Production: We’ve taken extreme care to source...,"[{'rewards': ""Pledge S$ 223 or more\n\nAbout $...",design/product design,20630.0,35000,2017-08-19 22:23:50,...,https://v2.kickstarter.com/1662714906-TVRNkAF2...,Automatik,https://www.kickstarter.com/profile/180642215,https://www.kickstarter.com/projects/180642215...,2017-07-12 04:41:53+00:00,2017-07-20 22:23:50+00:00,2017-07-20 22:23:50+00:00,https://www.kickstarter.com/projects/180642215...,"[{'rewards': ""Pledge S$ 223 or more About $16...",pledge s$ 223 or more about $160 1 watch -...
168,1807342594,SEAMS TAROT 3,South east asian Myths and Stories tarot,Story\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\...,With high freight prices and material prices; ...,[{'rewards': 'Pledge S$ 70 or more\n\nAbout US...,games/playing cards,28566.5,25000,2021-10-13 02:01:34,...,https://v2.kickstarter.com/1662683771-WTvgpayS...,Rowen,https://www.kickstarter.com/profile/seamstarot,https://www.kickstarter.com/projects/seamstaro...,2021-06-26 03:55:17+00:00,2021-09-13 02:01:34+00:00,2021-09-13 02:01:34+00:00,https://www.kickstarter.com/projects/seamstaro...,[{'rewards': 'Pledge S$ 70 or more About US$ ...,pledge s$ 70 or more about us$ 50 single d...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
582,967764342,Pikachu & Eevee Sweet Days Enamel Pin,This project features a set of 8 cute Pikachu ...,StoryINTRODUCTION\r\n\r\n\r\nHi friends! My na...,There are always going to be risks when it com...,[{'rewards': 'Pledge S$ 14 or more\n\nAbout U

In [131]:
#tokenize
tokenizer = TreebankWordTokenizer()
X_train["rewards_tokens"] = X_train["rewards_processed"].apply(lambda x : tokenizer.tokenize(x))

# remove stopwords

stop = stopwords.words('english')
custom_stopwords = ['$']
for word in custom_stopwords:
    stop.append(word)
    
X_train["rewards_tokens"] = X_train["rewards_tokens"].apply(lambda x : [word for word in x if word not in stop])


In [132]:
# POS tagging
from nltk.corpus import wordnet

def get_part_of_speech_tags(token):
    
    """Maps POS tags to first character of the pos tagging returned by pos_tag.
    We are focussing on Verbs, Nouns, Adjectives and Adverbs; otherwise return NOUN"""

    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    
    tag = nltk.pos_tag([token])[0][1][0].upper()
    
    return tag_dict.get(tag, wordnet.NOUN)

# lemmatize
lemmatizer = WordNetLemmatizer()
X_train["rewards_tokens_lemmatized_POS"] = X_train["rewards_tokens"].apply(lambda x : [lemmatizer.lemmatize(word, get_part_of_speech_tags(word)) for word in x])


In [133]:
# n-gram
X_train["rewards_tokens_lemmatized_POS_bigrams"] = X_train["rewards_tokens_lemmatized_POS"].apply(lambda x : list(ngrams(x, 2)))
X_train["rewards_tokens_lemmatized_POS_trigrams"] = X_train["rewards_tokens_lemmatized_POS"].apply(lambda x : list(ngrams(x, 3)))


In [ ]:
# #stemming
# ps = PorterStemmer()
# X_train["rewards_tokens"] = X_train["rewards_tokens"].apply(lambda x : [ps.stem(word) for word in x])

In [151]:
# TFIDF vectorizer
vect = TfidfVectorizer(analyzer='word', ngram_range=(1,3), max_features=100, min_df=2)
tfidf_fit = vect.fit(pd.Series(X_train["rewards_processed"]))
rewards_tfidf_array = tfidf_fit.transform(pd.Series(X_train["rewards_processed"])).toarray()



In [152]:
rewards_tfidf_array

array([[0.        , 0.        , 0.3355382 , ..., 0.02876259, 0.06084114,
        0.05549788],
       [0.14348722, 0.        , 0.2490629 , ..., 0.02277319, 0.13548326,
        0.2361841 ],
       [0.        , 0.        , 0.33529971, ..., 0.08622645, 0.04559842,
        0.        ],
       ...,
       [0.05002156, 0.        , 0.        , ..., 0.30565295, 0.08816505,
        0.        ],
       [0.00810308, 0.        , 0.        , ..., 0.04726267, 0.07141008,
        0.        ],
       [0.04059413, 0.        , 0.        , ..., 0.        , 0.25042087,
        0.13053036]])